In [2]:
import mcnpy
import os
from mcnpy.utils import configure_ace_debug_logging

# Configure debug logging
#configure_ace_debug_logging(enable=True)  # Set to False to disable debug output

In [3]:
# Define all ACE files to test
acefile = 'files/10010_71.02c'
acefile_U235 = 'files/922350_71.02c'
acefile1 = 'files/FE56.293600c'
acefile2 = 'files/260560_80.02c'

# Print files to verify they exist
for file_path in [acefile, acefile_U235, acefile1, acefile2]:
    print(f"File {file_path} exists: {os.path.exists(file_path)}")

File files/10010_71.02c exists: True
File files/922350_71.02c exists: True
File files/FE56.293600c exists: True
File files/260560_80.02c exists: True


In [4]:
ace2 = mcnpy.read_ace(acefile2)

In [5]:
ace2.mt_numbers

[1,
 2,
 5,
 16,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 91,
 101,
 102,
 103,
 107,
 111,
 203,
 204,
 205,
 206,
 207,
 444,
 600,
 601,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 609,
 649,
 800,
 801,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 814,
 815,
 816,
 817,
 818]

In [5]:
print(ace_U235.mt_numbers)

[1, 2, 4, 16, 17, 18, 37, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 101, 102, 444]


In [6]:
ace_U235.cross_section.reaction.keys()

dict_keys([1, 2, 101, 16, 17, 18, 37, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 102, 444, 4])

In [7]:
print(ace_U235.cross_section.mt_numbers)

[1, 2, 4, 16, 17, 18, 37, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 101, 102, 444]


In [8]:
ace_U235.cross_section.reaction[4].to_dataframe()

,Energy,Cross Section
0,0.00225,0.000000e+00
1,0.00225,1.204971e-07
2,0.00230,1.251350e-07
3,0.00240,1.342625e-07
4,0.00250,1.433900e-07
...,...,...
425,18.00000,4.111469e-01
426,18.50000,3.994417e-01
427,19.00000,3.882021e-01
428,19.50000,3.764036e-01


In [9]:
ace_U235.cross_section.to_dataframe()

,Energy,MT=1,MT=2,MT=4,MT=16,MT=17,MT=18,MT=37,MT=51,MT=52,...,MT=85,MT=86,MT=87,MT=88,MT=89,MT=90,MT=91,MT=101,MT=102,MT=444
0,1.000000e-11,37407.199500,58.331490,0.000000,0.000000,0.000000,31390.430000,0.000000e+00,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5958.438000,5958.438000,220.971000
1,1.031250e-11,36836.060100,57.493120,0.000000,0.000000,0.000000,30911.110000,0.000000e+00,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5867.457000,5867.457000,217.596900
2,1.062500e-11,36290.312800,56.692750,0.000000,0.000000,0.000000,30453.100000,0.000000e+00,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5780.520000,5780.520000,214.372700
3,1.093750e-11,35768.122700,55.927650,0.000000,0.000000,0.000000,30014.860000,0.000000e+00,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5697.335000,5697.335000,211.287800
4,1.125000e-11,35267.846300,55.195330,0.000000,0.000000,0.000000,29595.010000,0.000000e+00,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5617.641000,5617.641000,208.332300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76520,1.800000e+01,6.124479,3.134972,0.411147,0.211427,0.308173,2.058403,3.810030e-07,0.0,0.0,...,0.003198,0.001200,0.001310,0.001362,0.001021,0.001134,0.123749,0.000357,0.000357,0.219410
76521,1.850000e+01,6.147738,3.170238,0.399442,0.193505,0.340188,2.043946,8.441785e-05,0.0,0.0,...,0.003089,0.001158,0.001265,0.001315,0.000985,0.001095,0.119950,0.000335,0.000335,0.222942
76522,1.900000e+01,6.159560,3.190453,0.388202,0.178435,0.370183,2.031731,2.387410e-04,0.0,0.0,...,0.002983,0.001119,0.001222,0.001269,0.000951,0.001057,0.116218,0.000317,0.000317,0.226554
76523,1.950000e+01,6.165319,3.206368,0.376404,0.165893,0.393329,2.021675,1.348763e-03,0.0,0.0,...,0.002881,0.001081,0.001180,0.001226,0.000919,0.001021,0.111520,0.000302,0.000302,0.229946


In [10]:
# Filter df to show values in the specified energy range
energy_range = df[(df['Energy'] >= 2.249703e-3) & (df['Energy'] <= 3e-3)]
display(energy_range)

# Optional: Show how many rows are in this range
print(f"Number of points in this energy range: {len(energy_range)}")

NameError: name 'df' is not defined

In [ ]:
loc = 459766
print((loc)/4+13-0.25)

114954.25
